In [1]:
import mysql.connector
import re
import sshtunnel

In [2]:
with sshtunnel.open_tunnel(
    ssh_address_or_host=("htz3.host.fraam.de", 1022),
    remote_bind_address=("192.168.100.15", 22),
    ssh_username="root",
) as tun_htz3:

    with sshtunnel.open_tunnel(
        ssh_address_or_host=("localhost", tun_htz3.local_bind_port),
        remote_bind_address=("127.0.0.1", 3306),
        ssh_username="root",
    ) as tun_wpjail:

        with mysql.connector.connect(
            user="root",
            host="localhost",
            database="wordpress",
            port=tun_wpjail.local_bind_port,
        ) as conn:

            with conn.cursor() as cursor:
                cursor.execute(
                    "select ID, post_title, post_content from wp_posts where post_type = 'wpcf7_contact_form'"
                )
                data = cursor.fetchall()

In [3]:
parsed = [
    (d[0], d[1], re.search(r"wordpress@fraam.de\n(.+)", d[2]).group(1))
    for d in data
    if "wordpress@fraam.de" in d[2]
]
parsed

[(11, 'Contact form - Innovation', 'earl.kuehne@fraam.de'),
 (2328,
  'Contact form - Transformation - Kulturwandel',
  'nicol.huckfeldt@fraam.de'),
 (2329,
  'Contact form - Transformation - Digitale Kanzlei',
  'susanne.halmburger@fraam.de'),
 (2330, 'Contact form - Prozesse', 'klaus.stammerjohann@fraam.de'),
 (2331, 'Contact form - Technologie', 'enno.richter@fraam.de'),
 (2332, 'Contact form - Pragmatic Agile', 'earl.kuehne@fraam.de')]

In [4]:
gomap = (
    """package main

func GetRcpts() map[int]string {
\treturn map[int]string{\n"""
    + "\n".join([f'\t\t{x[0]}: "{x[2]}",' for x in parsed])
    + "\n\t}\n}"
)
print(gomap)

package main

func GetRcpts() map[int]string {
	return map[int]string{
		11: "earl.kuehne@fraam.de",
		2328: "nicol.huckfeldt@fraam.de",
		2329: "susanne.halmburger@fraam.de",
		2330: "klaus.stammerjohann@fraam.de",
		2331: "enno.richter@fraam.de",
		2332: "earl.kuehne@fraam.de",
	}
}


In [5]:
with open("rcpts.go", "w") as f:
    f.write(gomap)